# Laboratorio 9 - Deep Learning

### Autores

- Angel Higueros 20460
- Fredy Velasquez 201011

## Task 1 - Práctica

### Preparación de datos

## Task 2 - Teoria 

1. ¿Cuál es el problema del gradiente de fuga en las redes LSTM y cómo afecta la efectividad de LSTM para el pronóstico de series temporales?

El problema del gradiente de fuga en las redes LSTM se refiere a la tendencia de los gradientes durante la retropropagación a volverse muy pequeños o muy grandes a medida que se propagan hacia atrás en el tiempo a través de muchas unidades de tiempo en una secuencia larga. Esto puede dificultar el entrenamiento efectivo de modelos LSTM para series temporales, ya que los gradientes pequeños pueden llevar a un entrenamiento lento o a un estancamiento, mientras que los gradientes grandes pueden provocar explosiones del gradiente. Esto afecta la efectividad de LSTM para pronósticos de series temporales, ya que puede dificultar la captura de relaciones a largo plazo en los datos.

2. ¿Cómo se aborda la estacionalidad en los datos de series temporales cuando se utilizan LSTM para realizar pronósticos y qué papel juega la diferenciación en el proceso?
3. ¿Cuál es el concepto de "tamaño de ventana" en el pronóstico de series temporales con LSTM y cómo afecta la elección del tamaño de ventana a la capacidad del modelo para capturar patrones a corto y largo plazo?